# AI Community @ Семинар № 6, весна 2019
## Детектирование изображений

**Детектирование изображений** - задача компьютерного зрения, при которой необходимо найти объект определённого класса и показать область на изображении, где этот объект находится

![](https://cdn-images-1.medium.com/max/2000/1*95lJePt-70PH3PoVfz2yYQ.png)
![](https://cdn-images-1.medium.com/max/2000/1*tqbGt2zj-9OP_rjR57r8Zg.png)

# Regions-Convolutional Neural Networks (R-CNN)

### R-CNN — 2014

![](https://cdn-images-1.medium.com/max/1000/1*REPHY47zAyzgbNKC6zlvBQ.png)
1. Отбор кандидатов с помощью [Selective Search](https://ivi.fnwi.uva.nl/isis/publications/2013/UijlingsIJCV2013/UijlingsIJCV2013.pdf)
2. [AlexNet](http://vision.stanford.edu/teaching/cs231b_spring1415/slides/alexnet_tugce_kyunghee.pdf), но можно использовать и другой экстрактор признаков
3. [SVM](https://towardsdatascience.com/support-vector-machine-introduction-to-machine-learning-algorithms-934a444fca47) для каждого класса
4. Линейная регрессия для уточнения рамок


#### Selective Search
Выбор множества мелких похожих регионов и объединение похожих соседей
<img src='images/selective_search.png'>


#### Проблемы R-CNN:
1. Очень медленная модель, так как нужно для каждого региона делать прямой проход по свёрточной сети (регионов в среднем 2000 на одной картинке)
2. Результат алгоритма Selective Search никак не меняется в процессе обучения, из-за чего на вход модели могут поступать регионы плохого качества
3. Нужно учить три модели

### Fast R-CNN — 2015

Основная идея осталось такой же, но теперь вместо подачи областей кандидатов подаётся всё изображение один раз
1. Из выхода последнего сверточного слоя определяем регионы, где может быть объект, и превращает их в квадраты используя [RoI-Pool](https://deepsense.ai/region-of-interest-pooling-explained/), чтобы его можно было подавать в полностью связанный слой
2. Заменить SVM на слой Softmax
3. Заменить линейную регрессию на полносвязный слой

<img src='images/fast_rcnn.png'>

![](https://cdn-images-1.medium.com/max/2000/1*m2QO_wbUPA05mY2q4v7mjg.png)
Значительная разница в скорости обучения и прогона между R-CNN и Fast R-CNN

Но видно, что ещё осталась проблема: медленный отбор регионов-кандидатов.

### Faster R-CNN — 2016

Раз у нас медленный алгоритм генерации регионов-кандидатов, то можно заменить его сетью, которая на вход принимает выходы со свёрточной сети.

<img src='images/faster_rcnn.png'>

#### Region proposal framework
В RPN по извлечённым CNN признакам скользят свёрткой с окном 3х3. Полученные с её помощью значения передаются в два параллельных полносвязанных слоя: box-regression layer (reg) и box-classification layer (cls).
Для того, чтобы разделять признаки, получаемые в CNN, между RPN и модулем детектирования, процесс обучения всей сети построен итерационно, с использованием нескольких шагов:

1. Инициализируется и обучается на определение регионов-кандидатов RPN-часть.
2. С использованием предлагаемых RPN регионов заново обучается Fast R-CNN часть.
3. Обученная сеть детектирования используется, чтобы инициализировать веса для RPN. Общие convolution-слои, однако, фиксируются и производится донастройка только слоёв, специфичных для RPN.
4. С зафиксированными convolution-слоями окончательно донастраивается Fast R-CNN.

<img src='images/region_proposal.png'>

Проблема: хотим не рамки объекта, хотим сам объект!

### Mask R-CNN — 2017

Mask R-CNN развивает архитектуру Faster R-CNN путём добавления ещё одной ветки, которая предсказывает положение маски, покрывающей найденный объект, и, таким образом, решает уже задачу instance segmentation

<img src='images/mask_rcnn.png'>

<img src='images/mask_rcnn_example.png'>

Лосс-функция состоиз из суммы ошибок на классификаторе, ошибки определения границы и ошибки маски

 $$ L = L_{cls} + L_{box} + L_{mask} $$

Одна из основных модификаций, возникших из-за необходимости предсказывать маску — изменение процедуры RoIPool(вычисляющей матрицу признаков для региона-кандидата) на так называемую RoIAlign. В RoIPool проблема решалась просто округлением дробных значений до целых. В противоположность этому, в RoIAlign не используется округление, все числа остаются действительными, а для вычисления значений признаков используется билинейная интерполяция по четырём ближайшим целочисленным точкам. 

![](https://habrastorage.org/webt/ml/27/mq/ml27mqsszmzpo4-j0iv-yobw1os.png)

### Имплементации

**Faster R-CNN**

PyTorch: https://github.com/longcw/faster_rcnn_pytorch  
TensorFlow: https://github.com/smallcorgi/Faster-RCNN_TF  

**Mask R-CNN**

PyTorch: https://github.com/felixgwu/mask_rcnn_pytorch  
TensorFlow: https://github.com/CharlesShang/FastMaskRCNN  

### Ссылки

Статья для людей: https://blog.athelas.com/a-brief-history-of-cnns-in-image-segmentation-from-r-cnn-to-mask-r-cnn-34ea83205de4

Статья на русском: https://habr.com/ru/post/421299/

Статьи с архива:
1. R-CNN: https://arxiv.org/abs/1311.2524
1. Fast R-CNN: https://arxiv.org/abs/1504.08083
1. Faster R-CNN: https://arxiv.org/abs/1506.01497
1. Mask R-CNN: https://arxiv.org/abs/1703.06870

# You Only Look Once (YOLO)

## YOLOv1 — 2016

Основная идея заключается в том, что вместо того, чтобы иметь несколько разных сетей, обучать одну сеть, которая делает всё сразу.
![](https://cdn-images-1.medium.com/max/800/1*9nikM2b0u-m67SJpQXftKA.png)

#### Обработка изображения происходит следующим способом:

1. Входное изображение разбивется на сетку S×S (S=7).
2. Каждая ячейка сетки предсказывает B ограничивающих рамок (B=2) и оценки уверенности для этих рамок. Оценка уверенности показывает, насколько модель уверена в том, что в рамке есть объект.
3. Каждая ограничивающая рамка состоит из 5 предсказаний: x, y, w, h и уверенности. x, y - координаты центра. w, h - ширина и высота рамки.
4. Каждая ячейка предсказывает условную вероятность класса объекта для N (N=20) классов.

![](https://cdn-images-1.medium.com/max/800/1*YG6heD55fEmZeUKRSlsqlA.png)

#### Архитектура модели

![](https://cdn-images-1.medium.com/max/1400/1*q5feieizWKYq7dpWjYvCOw.png)

Модель состоит из 24 сверточных слоев, за которыми следуют 2 полносвязных слоя. Чередующиеся сверточные слои 1×1 уменьшают пространство объектов от предыдущих слоев.

Так же существует быстрая версия этой модели с 9 слоями и уменьшенным количеством фильтров в них.

#### Функция ошибки

![](https://cdn-images-1.medium.com/max/1200/1*1noFJp_rnXsG0TJvtoghFA.png)

$\lambda_{noobj}$ - коэффициент, необходимый, чтобы сеть сильно не переобучалась напримерах, когда объекта нет. В оригинальной статье равер 0.5

$\lambda_{coord}$ - коэффициент, необходимый, чтобы сеть лучше выбирала рамки. В оригинальной статье равер 5.

#### Дополнительные детали

1. На всех слоях, за исключением последнего, использовалась leaky ReLU функция активации
1. Первые 20 слоёв были предобучены на ImageNet до 88% top-5 accuracy.
1. Затем вся сеть обучалась 135 эпох, используя PASCAL VOC 2007 and 2012 в качестве датасета.
1. Для того, чтобы исключить множественные перекрывающиеся рамки используется Non-max suppression.

#### Сравнение с другими моделями (R-CNN)

![](https://cdn-images-1.medium.com/max/800/1*zxDddLucPHk2sM5DHHEWvw.png)

#### Немного примеров работы
<img src='images/predictions.bmp'>

## YOLOv2 & YOLO9000 — 2016

В конце того же года выходит статья "YOLO9000: Better, Faster, Stronger", в которой показаны множественные улучшения первой версии.

### Улучшения для YOLOv2 (Better) :

1. [Batch Normalization (BN)](https://medium.com/@sh.tsang/review-batch-normalization-inception-v2-bn-inception-the-2nd-to-surpass-human-level-18e2d0f56651)
1. Классификатор более высокого разрешения. После обучения на изображениях с разрешением 224×224 , YOLOv2 также использует изображения с разрешением 448×448 для тонкой настройки классификатора для 10 эпох на ImageNet
1. Предсказание более точного местоположения. Так первая версия не имела ограничений на предсказываемых рамках из-за чего была нестабильна.
![](https://cdn-images-1.medium.com/max/900/1*r_LqJNwlbapcFbSc8lytRg.png)
1. Изменение размера карты признаков с 7×7 до 13×13 и конкатенирование его с предыдущем слоем размера 26×26×512 изменённого в 13×13×2024
1. Обучение на масштабированных изображениях. Размеры изображений - {320, 352, …, 608}
    
#### Результат
![image.png](https://cdn-images-1.medium.com/max/1200/1*2JHsliX3aGSP5QZznsa8pg.png)

![](https://cdn-images-1.medium.com/max/800/1*DWN1vM5yUtqt1gIjTdiUeg.png)

### YOLOv2 - Darknet-19 (Faster)

Помимо всех указанных выше изменений поменялась так же архитектура сети
![](https://cdn-images-1.medium.com/max/800/1*iPHGuCWfCOTjrEW187fSZQ.png)

Сравнение с другими архитектурами, используемыми в YOLOv2
![](https://cdn-images-1.medium.com/max/800/1*XNpkCCTVfEDfmGzk5VkL1g.png)

### YOLO9000 - WordTree (Stronger) :
У нас есть много разных даатесов для класификации и детектирования. Почему бы их не объединить?

![](https://cdn-images-1.medium.com/max/1200/1*1rpDaEiL-4NuTBlk9p0oAg.png)

Microsoft COCO: 100 тысяч изображений, 80 классов, лейблы для детектирования, классы в основном имеют обобщённые понятия, такие как “dog” or “boat”.

ImageNet: 13 миллионов изображений, 22 тысячи классов, лейблы для классификации, классы обозначают узкие понятия, такие как “Norfolk terrier”, “Yorkshire terrier”, или “Bedlington terrier”

При этом классы “dog” и “Norfolk terrier” не взаимоисключающие.

Поэтому авторы объединили классы в дерево слов (WordTree)
![](https://cdn-images-1.medium.com/max/800/1*YiX61mdylOzZYlBFXl9HjA.png)

В результате получилась модель, детектирующая очень специфичные классы
<img src='images/prediction_yolo9000.bmp'>

## YOLOv3 — 2018

Спустя некоторое время была опубликована третья версия детектора YOLO с новым экстрактором признаков и ещё несколькими изменениями

### Предсказание классов

Вместо Softmax используются независимые логистические классификаторы и binary cross-entropy. Это сделано потому, что могут быть перекрывающиеся метки для классификации нескольких меток, например, если YOLOv3 на более сложном наборе данных, например, Open Images Dataset.

### Предсказание через масштабирование

1. Используются три разных масштаба
2. Признали извлекаются используя [Feature Pyramid Network](https://towardsdatascience.com/review-fpn-feature-pyramid-network-object-detection-262fc7482610)
3. Несколько дополнительных слоёв добавлено в экстрактор признаков Darknet-53.
4. Последние три слоя предсказывают параметры ограничивающей рамки, уверенность в том, что в области есть объект и метку класса.

### Darknet-53

![](https://cdn-images-1.medium.com/max/800/1*tF1fK8-D5PVDb4khxvIH_g.png)

1. Сеть также использует Batch normalization.
1. Присутствуют шорткаты, связывающие различные блоки вместе (как в ResNet).

![](https://cdn-images-1.medium.com/max/800/1*U8n4uNBiRzhK8ZYlHbHZpg.png)

### Результат 

![](https://cdn-images-1.medium.com/max/1080/1*6LvKy1Cb0SQ1PLE26QgBgQ.png)

### Ссылки

Статьи с объяснениями и красивыми картинками
1. [YOLOv1](https://towardsdatascience.com/yolov1-you-only-look-once-object-detection-e1f3ffec8a89)
1. [YOLOv2 & YOLO9000](https://towardsdatascience.com/review-yolov2-yolo9000-you-only-look-once-object-detection-7883d2b02a65)
1. [YOLOv3](https://towardsdatascience.com/review-yolov3-you-only-look-once-object-detection-eab75d7a1ba6)

Статьи с архива:
1. YOLOv1: https://arxiv.org/pdf/1506.02640v5.pdf
2. YOLO9000: https://arxiv.org/pdf/1612.08242.pdf
3. YOLOv3: https://arxiv.org/pdf/1804.02767.pdf

[Список других моделей для детектирования](https://paperswithcode.com/task/object-detection)